In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [5]:
data = pd.read_csv('train.csv')

In [6]:
data.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
data = np.array(data)
m, n = data.shape
np.random.shuffle(data) # shuffle before splitting into dev and training sets

data_dev = data[0:1000].T
Y_dev = data_dev[0]
X_dev = data_dev[1:n]
X_dev = X_dev / 255.

data_train = data[1000:m].T
Y_train = data_train[0]
X_train = data_train[1:n]
X_train = X_train / 255.
_,m_train = X_train.shape

In [10]:
X_train[:0].shape

(0, 0)

In [11]:
X_train[0:].shape

(784, 0)

In [26]:
def init_params():
    W1 = np.random.rand(10, 784) - 0.5
    b1 = np.random.rand(10, 1) - 0.5
    W2 = np.random.rand(10, 10) - 0.5
    b2 = np.random.rand(10, 1) - 0.5
    return W1, b1, W2, b2

def ReLU(Z):
    return np.maximum(Z, 0)

def softmax(Z):
    A = np.exp(Z) / sum(np.exp(Z))
    return A

def forward_prop(W1, b1, W2, b2, X):
    Z1 = W1.dot(X) + b1
    A1 = ReLU(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = softmax(Z2)
    return Z1, A1, Z2, A2

def ReLU_deriv(Z):
    return Z > 0

def one_hot(Y):
    if Y.size == 0:
        raise ValueError("Y cannot be empty")
    Y = Y.astype(int)
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

def backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y):
    one_hot_Y = one_hot(Y)
    dZ2 = A2 - one_hot_Y
    dW2 = 1 / m * dZ2.dot(A1.T)
    db2 = 1 / m * np.sum(dZ2)
    dZ1 = W2.T.dot(dZ2) * ReLU_deriv(Z1)
    dW1 = 1 / m * dZ1.dot(X.T)
    db1 = 1 / m * np.sum(dZ1)
    return dW1, db1, dW2, db2

def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1
    W2 = W2 - alpha * dW2
    b2 = b2 - alpha * db2
    return W1, b1, W2, b2

In [27]:
def get_predictions(A2):
    return np.argmax(A2, 0)

def get_accuracy(predictions, Y):
    print(predictions, Y)
    return np.sum(predictions == Y) / Y.size

def gradient_descent(X, Y, alpha, iterations):
    W1, b1, W2, b2 = init_params()
    for i in range(iterations):
        Z1, A1, Z2, A2 = forward_prop(W1, b1, W2, b2, X)
        dW1, db1, dW2, db2 = backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y)
        W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)
        if i % 10 == 0:
            print("Iteration: ", i)
            predictions = get_predictions(A2)
            print(get_accuracy(predictions, Y))
    return W1, b1, W2, b2

In [31]:
W1, b1, W2, b2 = gradient_descent(X_dev, Y_dev, 0.1  , 500)

Iteration:  0
[2 2 9 2 5 7 4 5 2 1 9 4 9 2 6 1 2 4 9 6 4 4 2 2 2 9 9 4 7 4 2 3 4 3 1 6 4
 1 1 7 4 9 7 9 1 2 1 2 4 2 4 4 4 2 7 1 7 2 1 4 1 7 2 9 4 2 1 4 7 9 5 5 5 2
 1 4 5 4 2 5 4 4 4 6 1 2 1 4 1 1 2 2 7 2 7 4 6 4 1 7 2 1 2 1 1 4 4 7 2 4 2
 7 2 1 4 1 5 7 9 1 7 1 1 7 2 4 7 1 1 2 4 4 1 1 4 4 2 9 7 2 9 2 4 1 2 4 5 1
 2 4 7 1 4 7 9 4 2 1 4 4 9 1 1 4 1 1 1 4 4 4 1 1 7 4 9 1 2 1 4 1 1 4 7 1 1
 7 1 1 1 2 1 4 5 9 6 1 4 1 2 2 1 4 7 1 1 1 7 4 7 9 1 6 6 2 4 1 2 2 2 7 4 7
 7 7 4 1 7 2 1 4 4 7 7 7 2 4 1 4 1 1 1 1 4 2 2 1 2 4 4 4 1 7 6 4 4 1 7 1 4
 7 4 1 1 4 2 7 1 1 4 2 9 4 4 1 9 1 1 4 1 1 2 4 1 4 6 4 1 1 7 1 4 1 1 1 9 2
 9 7 9 2 2 7 1 5 3 4 5 4 7 4 6 1 1 7 1 7 1 1 1 7 9 1 4 2 1 3 1 1 2 2 2 4 4
 7 2 5 4 1 1 4 7 1 7 7 1 7 7 1 7 4 7 7 7 1 4 0 1 2 1 4 3 7 7 1 2 4 1 1 4 7
 1 1 4 1 2 1 6 7 7 1 4 4 4 4 2 4 7 2 1 7 4 4 4 4 1 9 2 9 2 4 5 4 7 7 4 1 4
 1 9 4 1 1 1 2 4 1 1 1 2 7 1 4 2 1 1 1 1 1 7 1 2 1 1 9 2 1 9 2 7 7 1 1 1 7
 7 4 4 9 1 2 4 1 1 1 6 1 1 7 4 1 2 9 4 1 2 1 1 1 4 4 9 4 4 7 6 4 2 4 1 4 2
 2 9 1 4 4 